In [1]:
import io
import re
import string
import tensorflow as tf
import tqdm

from tensorflow.keras import Model
from tensorflow.keras.layers import Dot, Embedding, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [2]:
%load_ext tensorboard

In [3]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [4]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

8


In [5]:
vocab, index = {}, 1
vocab['<pad>'] = 0
for token in tokens:
    if token not in vocab:
        vocab[token] = index
        index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


In [6]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


In [7]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


In [8]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
    example_sequence,
    vocabulary_size=vocab_size,
    window_size=window_size,
    negative_samples=0)

print(len(positive_skip_grams))

26


In [9]:
for target, context in positive_skip_grams[:5]:
    print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(4, 1): (shimmered, the)
(5, 1): (in, the)
(1, 6): (the, hot)
(5, 6): (in, hot)
(6, 5): (hot, in)


In [10]:
target_word, context_word = positive_skip_grams[0]

print(f"({target_word}, {context_word})::({inverse_vocab[target_word]}, {inverse_vocab[context_word]})")

num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1,1))

print(context_class)

negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,
    num_true=1,
    num_sampled=num_ns,
    unique=True,
    range_max=vocab_size,
    seed=SEED,
    name='negative_sampling')

print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])


(4, 1)::(shimmered, the)
tf.Tensor([[1]], shape=(1, 1), dtype=int64)
tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['wide', 'the', 'shimmered', 'road']


In [11]:
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

context = tf.concat([context_class, negative_sampling_candidates], 0)

print(context)

label = tf.constant([1] + [0]*num_ns, dtype="int64")

print(label)

target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

print(target, context, label)


tf.Tensor(
[[1]
 [2]
 [1]
 [4]
 [3]], shape=(5, 1), dtype=int64)
tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor(4, shape=(), dtype=int32) tf.Tensor([1 2 1 4 3], shape=(5,), dtype=int64) tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


In [12]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 4
target_word     : shimmered
context_indices : [1 2 1 4 3]
context_words   : ['the', 'wide', 'the', 'shimmered', 'road']
label           : [1 0 0 0 0]


In [13]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [14]:
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
    targets, contexts, labels = [], [], []
    sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

    for sequence in tqdm.tqdm(sequences):

        positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
            sequence,
            vocabulary_size=vocab_size,
            sampling_table=sampling_table,
            window_size=window_size,
            negative_samples=0)

        for target_word, context_word in positive_skip_grams:
            context_class = tf.expand_dims(
                tf.constant([context_word], dtype="int64"), 1)
            negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
                true_classes=context_class,
                num_true=1,
                num_sampled=num_ns,
                unique=True,
                range_max=vocab_size,
                seed=SEED,
                name='negative_sampling')
            
            negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1)

            context = tf.concat([context_class, negative_sampling_candidates], 0)
            label = tf.constant([1] + [0]*num_ns, dtype='int64')

            targets.append(target_word)
            contexts.append(context)
            labels.append(label)

    return targets, contexts, labels

In [15]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [16]:
with open(path_to_file) as f:
    lines = f.read().splitlines()
for line in lines[:20]:
    print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


In [17]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

In [18]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    return tf.strings.regex_replace(lowercase, '[%s]' % re.escape(string.punctuation), '')



In [19]:
vocab_size = 4096
sequence_length = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [20]:
vectorize_layer.adapt(text_ds.batch(1024))

In [21]:
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', 'the', 'and', 'to', 'i', 'of', 'you', 'my', 'a', 'that', 'in', 'is', 'not', 'for', 'with', 'me', 'it', 'be', 'your']


In [22]:
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

In [23]:
print(text_vector_ds)

<_UnbatchDataset shapes: (10,), types: tf.int64>


In [24]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

32777


In [25]:
for seq in sequences[:5]:
    print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => ['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', '', '']
[34  0  0  0  0  0  0  0  0  0] => ['all', '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => ['speak', 'speak', '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => ['first', 'citizen', '', '', '', '', '', '', '', '']


In [79]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=3,
    num_ns=7,
    vocab_size=vocab_size,
    seed=SEED)
print(len(targets), len(contexts), len(labels))

100%|██████████| 32777/32777 [00:06<00:00, 4744.65it/s]
88632 88632 88632


In [80]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000

dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset shapes: (((1024,), (1024, 8, 1)), (1024, 8)), types: ((tf.int32, tf.int64), tf.int64)>


In [81]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<PrefetchDataset shapes: (((1024,), (1024, 8, 1)), (1024, 8)), types: ((tf.int32, tf.int64), tf.int64)>


In [82]:
class Word2Vec(Model):
    def __init__(self, vocab_size, embedding_dim):
        super(Word2Vec, self).__init__()
        self.target_embedding = Embedding(vocab_size, embedding_dim, input_length=1, name='w2v_embedding')
        self.context_embedding = Embedding(vocab_size, embedding_dim, input_length=num_ns+1)
        self.dots = Dot(axes=(3,2))
        self.flatten = Flatten()

    def call(self, pair):
        target, context = pair
        word_emb = self.target_embedding(target)
        context_emb = self.context_embedding(context)
        dots = self.dots([context_emb, word_emb])
        return self.flatten(dots)

In [84]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam', loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [85]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")


In [86]:
word2vec.fit(dataset, epochs=100, callbacks=[tensorboard_callback])
print(word2vec.summary())


Epoch 1/100
86/86 [==============================] - 1s 11ms/step - loss: 2.0787 - accuracy: 0.1425
Epoch 2/100
86/86 [==============================] - 1s 9ms/step - loss: 2.0552 - accuracy: 0.3954
Epoch 3/100
86/86 [==============================] - 1s 10ms/step - loss: 1.9892 - accuracy: 0.3377
Epoch 4/100
86/86 [==============================] - 1s 10ms/step - loss: 1.8944 - accuracy: 0.3491
Epoch 5/100
86/86 [==============================] - 1s 11ms/step - loss: 1.7960 - accuracy: 0.3875
Epoch 6/100
86/86 [==============================] - 1s 10ms/step - loss: 1.7004 - accuracy: 0.4278
Epoch 7/100
86/86 [==============================] - 1s 8ms/step - loss: 1.6080 - accuracy: 0.4689
Epoch 8/100
86/86 [==============================] - 1s 9ms/step - loss: 1.5191 - accuracy: 0.5107
Epoch 9/100
86/86 [==============================] - 1s 8ms/step - loss: 1.4340 - accuracy: 0.5479
Epoch 10/100
86/86 [==============================] - 1s 8ms/step - loss: 1.3531 - accuracy: 0.5835
Epoc

In [87]:
%tensorboard --logdir logs


Reusing TensorBoard on port 6006 (pid 952), started 2:39:14 ago. (Use '!kill 952' to kill it.)

In [88]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [89]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()